In [1]:
import pandas as pd

In [2]:
ddi_subset = pd.read_csv("../mined_data/DDI_subset.csv")
dpi_subset = pd.read_csv("../mined_data/DPI_subset.csv")

In [5]:
len(ddi_subset), len(dpi_subset)

(500, 256)

In [22]:
list(filter(lambda x: x[1] > 1, ddi_subset["drug_1_name"].value_counts().to_dict().items()))

[('Salbutamol', 79),
 ('phenytoin', 57),
 ('fenofibrate', 52),
 ('lorazepam', 51),
 ('carisoprodol', 45),
 ('felodipine', 39),
 ('dipyridamole', 28),
 ('glimepiride', 28),
 ('leflunomide', 26),
 ('chlorthalidone', 17),
 ('meperidine', 15),
 ('Etiracetam', 13),
 ('tizanidine', 11),
 ('isoflurane', 7),
 ('minoxidil', 7),
 ('naratriptan', 7),
 ('zafirlukast', 6),
 ('Deprenyl', 5),
 ('caffeine', 2),
 ('Voriconazole', 2)]

In [28]:
from itertools import combinations

In [62]:
s = list(ddi_subset.query("drug_1_name == 'Salbutamol'").itertuples(index=False))
d = list(combinations(s, 2))

In [64]:
pd.DataFrame(d[0])

,drug_1_CID,drug_1_name,drug_1_SMILES,relationship,drug_2_CID,drug_2_name,drug_2_SMILES
0,CID000002083,Salbutamol,CC(C)(C)NCC(C1=CC(=C(C=C1)O)CO)O,abnormal ECG,CID000071616,Voriconazole,CC(C1=NC=NC=C1F)C(CN2C=NC=N2)(C3=C(C=C(C=C3)F)F)O
1,CID000002083,Salbutamol,CC(C)(C)NCC(C1=CC(=C(C=C1)O)CO)O,loss of weight,CID006918453,AURANOFIN,CCP(CC)CC.CC(=O)OCC1C(C(C(C(O1)[S-])OC(=O)C)OC...


In [3]:
from pathlib import Path
drug_data_path = Path("../background_information_data/drug_data")
protein_data_path = Path("../background_information_data/protein_data")

In [6]:
prompts = []
for row in ddi_subset.itertuples():
    with Path(drug_data_path / f"{row.drug_1_name}.txt").open("r") as drug_data_file:
        drug_1_data = max(drug_data_file.readlines(), key=len)

    with Path(drug_data_path / f"{row.drug_2_name}.txt").open("r") as drug_data_file:
        drug_2_data = max(drug_data_file.readlines(), key=len)

    triple = f"{row.drug_1_name} - {row.relationship} - {row.drug_2_name}"
    
    """if row.mode != row.action:
        triple = f"{row.drug_name} - {row.mode} and {row.action} - {row.protein_name}"
    else:
        triple = f"{row.drug_name} - {row.mode} - {row.protein_name}"
    """
    prompt = f"""You are given a drug-drug interaction and background information on each drug. 
I would like you to write 5 multiple-choice question-answer pairs utilizing this information. Each question must combine information about the drugs and their relationship.

Please structure your output as follows,
Question: <question text>
Choices: <answer choices labelled as (a), (b), (c), (d)>
Correct choice: <among (a), (b), (c) or (d)>

Drug 1 name: {row.drug_1_name} 
Drug 1 information: {drug_1_data.strip()}

Drug 2 name: {row.drug_2_name}
Drug 2 information: {drug_2_data.strip()}

Drug-Drug interaction triple: {triple}
"""
    prompts.append(prompt)

In [9]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4o")

In [26]:
cost = []
for prompt in prompts:
    input_tokens = len(enc.encode(prompt))
    input_cost = (input_tokens * 15) / 1_000_000
    output_cost = (60 * 1000) / 1_000_000
    cost.append(input_cost + output_cost)
sum(cost)

25.48134

In [17]:
len(enc.encode("""Question: What is the primary mechanism through which mirtazapine interacts with the HRH1 protein?
Choices:
(a) Activation of HRH1
(b) Inhibition of HRH1
(c) Enhancement of HRH1 expression
(d) No interaction with HRH1
Correct choice: (b) Inhibition of HRH1

Question: How might the inhibition of HRH1 by mirtazapine contribute to its antidepressant effects?
Choices:
(a) By reducing histamine-mediated neurotransmission, which can have sedative and anxiolytic effects
(b) By increasing serotonin reuptake, leading to greater synaptic serotonin availability
(c) By directly stimulating dopamine receptors in the prefrontal cortex
(d) By enhancing norepinephrine release through direct activation of adrenergic receptors
Correct choice: (a) By reducing histamine-mediated neurotransmission, which can have sedative and anxiolytic effects

Question: Given HRH1's role in histamine signaling, how might mirtazapine's inhibition of HRH1 affect side effects associated with its use?
Choices:
(a) Increased risk of insomnia due to reduced histamine signaling
(b) Increased sedation and drowsiness due to reduced histamine signaling
(c) Increased alertness and wakefulness due to enhanced histamine signaling
(d) Decreased appetite due to inhibition of histamine receptors
Correct choice: (b) Increased sedation and drowsiness due to reduced histamine signaling

Question: Considering HRH1's involvement in immune response and biofilm formation, what potential unintended effect might mirtazapine have in bacterial infections?
Choices:
(a) Mirtazapine may enhance bacterial biofilm formation by stimulating HRH1
(b) Mirtazapine may reduce bacterial biofilm formation due to HRH1 inhibition
(c) Mirtazapine has no known effect on bacterial infections
(d) Mirtazapine may increase bacterial resistance through HRH1 activation
Correct choice: (b) Mirtazapine may reduce bacterial biofilm formation due to HRH1 inhibition

Question: Based on the known role of HRH1 in both bacterial biofilm formation and histamine signaling, what dual benefit could mirtazapine provide in specific clinical contexts?
Choices:
(a) Treatment of depression while potentially reducing biofilm-associated infections
(b) Enhanced immune response against infections and improved cognitive function
(c) Increased neurotransmitter activity and direct bactericidal effects
(d) Reduced risk of allergies while increasing bacterial resistance
Correct choice: (a) Treatment of depression while potentially reducing biofilm-associated infections
"""))

535

In [ ]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Llama-3.3-70B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
    pad_token_id=tokenizer.eos_token_id
)

def generate_response(messages):
    input_text = ""
    for message in messages:
        input_text += f"{message['role']}: {message['content']}\n"
    
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=250, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip(input_text)

messages = [
    {"role": "system", "content": "You are a question answering AI assistant. I would like you to solve the following multiple-choice question. To assist you, I am providing you with the necessary background information. First, provide an explanation for your answer and then the answer as say, 'answer: a'. If there is no, please say 'answer not found'."},
    {"role": "user", "content": """Drug name: mirtazapine 
Drug information: BACKGROUND: Major depressive disorders have a significant impact on children and adolescents, including on educational and vocational outcomes, interpersonal relationships, and physical and mental health and well-being. There is an association between major depressive disorder and suicidal ideation, suicide attempts, and suicide. Antidepressant medication is used in moderate to severe depression; there is now a range of newer generations of these medications. OBJECTIVES: To investigate, via network meta-analysis (NMA), the comparative effectiveness and safety of different newer generation antidepressants in children and adolescents with a diagnosed major depressive disorder (MDD) in terms of depression, functioning, suicide-related outcomes and other adverse outcomes. The impact of age, treatment duration, baseline severity, and pharmaceutical industry funding was investigated on clinician-rated depression (CDRS-R) and suicide-related outcomes. SEARCH METHODS: We searched the Cochrane Common Mental Disorders Specialised Register, the Cochrane Library (Central Register of Controlled Trials (CENTRAL) and Cochrane Database of Systematic Reviews (CDSR)), together with Ovid Embase, MEDLINE and PsycINFO till March 2020. SELECTION CRITERIA: Randomised trials of six to 18 year olds of either sex and any ethnicity with clinically diagnosed major depressive disorder were included. Trials that compared the effectiveness of newer generation antidepressants with each other or with a placebo were included. Newer generation antidepressants included: selective serotonin reuptake inhibitors; selective norepinephrine reuptake inhibitors (SNRIs); norepinephrine reuptake inhibitors; norepinephrine dopamine reuptake inhibitors; norepinephrine dopamine disinhibitors (NDDIs); and tetracyclic antidepressants (TeCAs). DATA COLLECTION AND ANALYSIS: Two reviewers independently screened titles/abstracts and full texts, extracted data, and assessed risk of bias. We analysed dichotomous data as Odds Ratios (ORs), and continuous data as Mean Difference (MD) for the following outcomes: depression symptom severity (clinician rated), response or remission of depression symptoms, depression symptom severity (self-rated), functioning, suicide related outcomes and overall adverse outcomes. Random-effects network meta-analyses were conducted in a frequentist framework using multivariate meta-analysis. Certainty of evidence was assessed using Confidence in Network Meta-analysis (CINeMA). We used "informative statements" to standardise the interpretation and description of the results. MAIN RESULTS: Twenty-six studies were included. There were no data for the two primary outcomes (depressive disorder established via clinical diagnostic interview and suicide), therefore, the results comprise only secondary outcomes. Most antidepressants may be associated with a "small and unimportant" reduction in depression symptoms on the CDRS-R scale (range 17 to 113) compared with placebo (high certainty evidence: paroxetine: MD -1.43, 95% CI -3.90, 1.04; vilazodone: MD -0.84, 95% CI -3.03, 1.35; desvenlafaxine MD -0.07, 95% CI -3.51, 3.36; moderate certainty evidence: sertraline: MD -3.51, 95% CI -6.99, -0.04; fluoxetine: MD -2.84, 95% CI -4.12, -1.56; escitalopram: MD -2.62, 95% CI -5.29, 0.04; low certainty evidence: duloxetine: MD -2.70, 95% CI -5.03, -0.37; vortioxetine: MD 0.60, 95% CI -2.52, 3.72; very low certainty evidence for comparisons between other antidepressants and placebo). There were "small and unimportant" differences between most antidepressants in reduction of depression symptoms (high- or moderate-certainty evidence). Results were similar across other outcomes of benefit. In most studies risk of self-harm or suicide was an exclusion criterion for the study. Proportions of suicide-related outcomes were low for most included studies and 95% confidence intervals were wide for all comparisons. The evidence is very uncertain about the effects of mirtazapine (OR 0.50, 95% CI 0.03, 8.04), duloxetine (OR 1.15, 95% CI 0.72, 1.82), vilazodone (OR 1.01, 95% CI 0.68, 1.48), desvenlafaxine (OR 0.94, 95% CI 0.59, 1.52), citalopram (OR 1.72, 95% CI 0.76, 3.87) or vortioxetine (OR 1.58, 95% CI 0.29, 8.60) on suicide-related outcomes compared with placebo. There is low certainty evidence that escitalopram may "at least slightly" reduce odds of suicide-related outcomes compared with placebo (OR 0.89, 95% CI 0.43, 1.84). There is low certainty evidence that fluoxetine (OR 1.27, 95% CI 0.87, 1.86), paroxetine (OR 1.81, 95% CI 0.85, 3.86), sertraline (OR 3.03, 95% CI 0.60, 15.22), and venlafaxine (OR 13.84, 95% CI 1.79, 106.90) may "at least slightly" increase odds of suicide-related outcomes compared with placebo. There is moderate certainty evidence that venlafaxine probably results in an "at least slightly" increased odds of suicide-related outcomes compared with desvenlafaxine (OR 0.07, 95% CI 0.01, 0.56) and escitalopram (OR 0.06, 95% CI 0.01, 0.56). There was very low certainty evidence regarding other comparisons between antidepressants. AUTHORS' CONCLUSIONS: Overall, methodological shortcomings of the randomised trials make it difficult to interpret the findings with regard to the efficacy and safety of newer antidepressant medications. Findings suggest that most newer antidepressants may reduce depression symptoms in a small and unimportant way compared with placebo. Furthermore, there are likely to be small and unimportant differences in the reduction of depression symptoms between the majority of antidepressants. However, our findings reflect the average effects of the antidepressants, and given depression is a heterogeneous condition, some individuals may experience a greater response. Guideline developers and others making recommendations might therefore consider whether a recommendation for the use of newer generation antidepressants is warranted for some individuals in some circumstances. Our findings suggest sertraline, escitalopram, duloxetine, as well as fluoxetine (which is currently the only treatment recommended for first-line prescribing) could be considered as a first option. Children and adolescents considered at risk of suicide were frequently excluded from trials, so that we cannot be confident about the effects of these medications for these individuals. If an antidepressant is being considered for an individual, this should be done in consultation with the child/adolescent and their family/caregivers and it remains critical to ensure close monitoring of treatment effects and suicide-related outcomes (combined suicidal ideation and suicide attempt) in those treated with newer generation antidepressants, given findings that some of these medications may be associated with greater odds of these events. Consideration of psychotherapy, particularly cognitive behavioural therapy, as per guideline recommendations, remains important.

Protein name: HRH1
Protein information: Staphylococcus aureus poses a significant threat to human health due to its virulence and multidrug resistance. In addition, recalcitrant biofilm formation of S. aureus often results in chronic infection and the treatment tolerance toward the traditional antibiotics. Thus, the development of novel antimicrobial agents capable to inhibit or eradicate S. aureus biofilm formation does matter. Here, we demonstrated that clemastine showed slight bacteriostatic activity and enhanced the antibacterial activity of oxacillin against S. aureus. Moreover, the dramatic inhibition of biofilm formation was found in clinical S. aureus strains by clemastine. Clemastine inhibited the release of eDNA during the biofilm formation and decreased the S. aureus hemolytic activity. Moreover, the S. aureus SA113 treated with clemastine displayed the decreased transcriptional level of the biofilm formation relevant genes (fnbB, icaA, and icaB), virulence genes (hlg, hld, lukde, lukpvl, beta-PSM, delta-PSM, and cap5A), and the regulatory genes agrA. The proteomics analysis of SA113 treated with clemastine demonstrated the significant changes in levels of biofilm-related proteins (stress response regulators ClpB and GroS, ATP-binding proteins, and urease metabolism), virulence-related proteins (SspA, superantigen, and VWbp), and methicillin resistance-related proteins (glutamine metabolism). The genetic mutations on gdpP (cyclic di-AMP phosphodiesterase) were found in the clemastine-induced tolerant derivative isolate by whole-genome sequencing. Furthermore, the interaction between clemastine and GdpP protein was demonstrated by the molecular docking, gdpP overexpression experiment, and thermal stability assay. Conclusively, clemastine might exert its inhibitory effects against the biofilm formation and hemolysis in S. aureus through targeting GdpP protein. IMPORTANCE The biofilm formation, which protects bacteria from stresses, including antibiotics and host immune responses, can be commonly found in clinical S. aureus isolates worldwide. Treatment failure of traditional antibiotics in biofilm-associated S. aureus infections remains a serious challenge. The novel anti-biofilm drug is urgently needed to address the looming crisis. In this study, clemastine, which is a histamine receptor H1 (HRH1) antagonist, was found to have a novel role of the significant inhibition against the biofilm formation and hemolytic activity of S. aureus and enhanced antibacterial activity against S. aureus when used in combination with oxacillin by targeting the GdpP protein. The discovery of this study identified novel use and mechanism of action of clemastine as a potential anti-biofilm drug for clinical application for S. aureus infectious.

Drug-Protein interaction triple: mirtazapine - inhibition - HRH1

Question: Mirtazapine is known to interact with the histamine H1 receptor (HRH1) as part of its pharmacological profile. Given its role in modulating neurotransmission, which of the following best describes the nature of this interaction?

Choices:
(a) Competitive binding with HRH1, leading to receptor activation 
(b) Negative modulation of HRH1, reducing its downstream signaling effects 
(c) Allosteric enhancement of HRH1 function
(d) No direct influence on HRH1 activity"""},
]

r = generate_response(messages)